In [1]:
import sys  
import os
import pandas as pd
import io
sys.path.insert(0, '../../../../scripts/')
sys.path.insert(0, '../../../../dataset/')
import hate_column_converter
import embedding_reader
import custom_keras_metrics
import tensorflow as tf
import keras.backend as K
import os
import time
import gc
import glob
import wget
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing import text, sequence
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import metrics
from tqdm import tqdm

2023-11-12 18:10:47.450200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-12 18:10:47.450243: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
#wget.download("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip", out="../../../../embeddings/")

In [ ]:
#with zipfile.ZipFile("../../../../embeddings/glove_s300.zip", 'r') as zip_ref:
 #   zip_ref.extractall("../../../../embeddings/")

In [2]:
MAX_LENGTH = 300
dataframe = pd.read_csv('../../../../dataset/HateBR.csv')
new_dataframe = hate_column_converter.convert_to_binary(dataframe)
classes_zero = new_dataframe[new_dataframe['hate_speech'] == 0]
classes_one = new_dataframe[new_dataframe['hate_speech'] == 1]
classes_zero_undersampled = classes_zero.sample(len(classes_one))
dataframe_undersampled = pd.concat([classes_zero_undersampled, classes_one]) 
instagram_comments = dataframe_undersampled['instagram_comments']
y = dataframe_undersampled['hate_speech']

/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numbers_in_hate_speech[['col1', 'col2']] = lines_with_two_numbers_in_hate_speech['hate_speech'].str.split(',', expand=True)
/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numb

In [3]:
x = dataframe_undersampled['instagram_comments']

In [4]:
token = Tokenizer()
token.fit_on_texts(x)
sequences = token.texts_to_sequences(x)
x = pad_sequences(sequences,maxlen=MAX_LENGTH)
vocabulary_size = len(token.word_index)+1

In [5]:
embedding_vector = embedding_reader.read_glove_file()
embedding_matrix = np.zeros((vocabulary_size,MAX_LENGTH))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 4867/4867 [00:00<00:00, 386659.30it/s]


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [9]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocabulary_size,MAX_LENGTH,weights=[embedding_matrix],input_length=x.shape[1],trainable=False))
lstm_model.add(Dense(256))
lstm_model.add(Dropout(0.25))
lstm_model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
lstm_model.add(MaxPooling1D(pool_size=2))
lstm_model.add(LSTM(128, return_sequences=True))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy', custom_keras_metrics.f1_score])

In [10]:
model = lstm_model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/20


2023-11-12 18:12:52.148168: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-11-12 18:12:52.195936: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39321600 exceeds 10% of free system memory.
2023-11-12 18:12:52.562960: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39321600 exceeds 10% of free system memory.
2023-11-12 18:12:52.636997: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39321600 exceeds 10% of free system memory.
2023-11-12 18:12:52.657510: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39321600 exceeds 10% of free system memory.


10/10 [==============================] - 12s 843ms/step - loss: 0.6930 - accuracy: 0.5113 - binary_accuracy: 0.5113 - val_loss: 0.6919 - val_accuracy: 0.5011 - val_binary_accuracy: 0.5011
Epoch 2/20
10/10 [==============================] - 8s 765ms/step - loss: 0.6916 - accuracy: 0.5134 - binary_accuracy: 0.5134 - val_loss: 0.6906 - val_accuracy: 0.5007 - val_binary_accuracy: 0.5007
Epoch 3/20
10/10 [==============================] - 7s 710ms/step - loss: 0.6910 - accuracy: 0.5131 - binary_accuracy: 0.5131 - val_loss: 0.6910 - val_accuracy: 0.5022 - val_binary_accuracy: 0.5022
Epoch 4/20
10/10 [==============================] - 7s 702ms/step - loss: 0.6904 - accuracy: 0.5159 - binary_accuracy: 0.5159 - val_loss: 0.6901 - val_accuracy: 0.5022 - val_binary_accuracy: 0.5022
Epoch 5/20
10/10 [==============================] - 7s 722ms/step - loss: 0.6883 - accuracy: 0.5159 - binary_accuracy: 0.5159 - val_loss: 0.6887 - val_accuracy: 0.5031 - val_binary_accuracy: 0.5031
Epoch 6/20
10/10 [==

In [ ]:
results = lstm_model.evaluate(x_test, y_test, batch_size=128)

for name, value in zip(lstm_model.metrics_names, results):
  print(name, ': ', value)
print()

In [ ]:
#import joblib

#model_filename = '../../../models/generated-models-base-paper/offensive_lstm_embeddings_model.pkl'
#joblib.dump(model, model_filename)